In [27]:
import nflgame
import numpy as np
import pandas as pd
import os
import datetime
import requests
import json

directory = '//users/cmirwin/desktop/Capstone/'

if not os.path.exists(directory):
    os.makedirs(directory)

    
#Import Team Info
dfTeamInfo = pd.read_csv(directory + 'TeamInfo.csv', sep=',')

#import Weather Info
dfWeatherInfo = pd.read_csv(directory + 'APIDataFinal.csv', sep=',') 

weeks = range(1, 17)

years = range(2009, 2017)

gamesPlayed = pd.DataFrame(columns=('GameID','Year', 'Week', 'HomeTeam', 'AwayTeam'))

GameId = 0

#Get all Games and Opponents to pull data for each game. 
for year in years:
    for week in weeks:
        games = nflgame.games(year, week = week)
        for game in games:
            gamesPlayed.loc[GameId]= (GameId + 1, year, week, game.home, game.away)
            GameId = GameId + 1
            
dfAllGames = gamesPlayed

dfNFLDataForGames = pd.DataFrame(columns = ('GameID', 'HomeTeam', 'AwayTeam', 'GameWinner', 'GameLoser', 'ScoreHome', 'ScoreAway', 'HomeFirstDowns'
    , 'HomePassingYards', 'HomePenaltyCnt', 'HomePenaltyYards', 'HomePossTime', 'HomePuntAvg', 'HomepuntCnt', 'HomePuntYards'
    , 'HomeRushingYards', 'HomeTotalYards', 'HomeTurnOvers', 'AwayFirstDowns', 'AwayPassingYards', 'AwayPenaltyCnt'
    , 'AwayPenaltyYards', 'AwayPossTime', 'AwayPuntAvg', 'AwaypuntCnt', 'AwayPuntYards', 'AwayRushingYards', 'AwayTotalYards'
    , 'AwayTurnOvers'))

dfScores = pd.DataFrame(columns=('GameID', 'Team', 'Quarter', 'ScoreType'))

dfPassingData = pd.DataFrame(columns=('GameID', 'Team', 'PassingComp', 'PassingAtt', 'PassingYards', 'PassingTD'))

dfRushingData = pd.DataFrame(columns=('GameID', 'Team', 'RushingAtt', 'RushingYards', 'RushingTD'))

print('DataFrames Created')

DataFrames Created


In [28]:
for x in range(0,len(dfAllGames)):
    gameId = dfAllGames.loc[x]['GameID']
    Year = dfAllGames.loc[x]['Year']
    Week = dfAllGames.loc[x]['Week']
    HomeTeam = dfAllGames.loc[x]['HomeTeam']
    AwayTeam = dfAllGames.loc[x]['AwayTeam']

    game = nflgame.one(Year, Week, HomeTeam, AwayTeam)

    try:
        homeStats = game.stats_home
        awayStats = game.stats_away

        dfNFLDataForGames.loc[x]= (gameId, HomeTeam, AwayTeam, game.winner, game.loser,  game.score_home, game.score_away
        , homeStats.first_downs, homeStats.passing_yds, homeStats.penalty_cnt, homeStats.penalty_yds
        , homeStats.pos_time, homeStats.punt_avg, homeStats.punt_cnt, homeStats.punt_yds
        , homeStats.rushing_yds, homeStats.total_yds, homeStats.turnovers, awayStats.first_downs
        , awayStats.passing_yds, awayStats.penalty_cnt, awayStats.penalty_yds, awayStats.pos_time
        , awayStats.punt_avg, awayStats.punt_cnt, awayStats.punt_yds, awayStats.rushing_yds
        , awayStats.total_yds, awayStats.turnovers)

    except:
        print("Error GameId:" , gameId)

print('Game Data Collected')

('Error GameId:', 1685)
Game Data Collected


In [33]:
for x in range(0,len(dfAllGames)):
    gameId = dfAllGames.loc[x]['GameID']
    Year = dfAllGames.loc[x]['Year']
    Week = dfAllGames.loc[x]['Week']
    HomeTeam = dfAllGames.loc[x]['HomeTeam']
    AwayTeam = dfAllGames.loc[x]['AwayTeam']

    game = nflgame.one(Year, Week, HomeTeam, AwayTeam)

    #find Scores foreach file
    try:
        scores = game.scores

        for score in scores:
            scoreSplit = score.split(' - ')

            dfScores.loc[(dfScores.GameID.count() + 1)] =(gameId,scoreSplit[0], scoreSplit[1], scoreSplit[2])

    except:
        print("Missing Drive Data")

print('Scoring Drives Captured')

dfScoreTotals = pd.DataFrame(dfScores.groupby(['GameID', 'Team', 'ScoreType'])['ScoreType'].count())

#Create Field Goals
dfFieldGoalTotal = dfScores[dfScores['ScoreType'] == 'FG']

#Create touchdowns
dfTouchdownTotal = dfScores[dfScores['ScoreType'] == 'TD']

maxGameID = max(dfTouchdownTotal['GameID']) + 1

#Create new column
dfNFLDataForGames['HomeFieldGoals'] = 0


dfNFLDataForGames['AwayFieldGoals'] = 0


for value in range(1, maxGameID):
    dfTeams = dfFieldGoalTotal[dfFieldGoalTotal['GameID'] == value]
    dfFieldGoals = dfTeams.groupby(['GameID', 'Team'])['ScoreType'].count().reset_index()
    run = 0
    for field in dfFieldGoals.values:
        if run == 0:
            dfNFLDataForGames.loc[value - 1, 'HomeFieldGoals'] = field[2]
            run = run + 1
        else:
            dfNFLDataForGames.loc[value -1 , 'AwayFieldGoals'] = field[2]

            
#Create Touchdowns
dfNFLDataForGames['HomeTouchDowns'] = 0

dfNFLDataForGames['AwayTouchDowns'] = 0
            
for value in range(1, maxGameID):
    dfTeamsTD = dfTouchdownTotal[dfTouchdownTotal['GameID'] == value]
    dfTD = dfTeamsTD.groupby(['GameID', 'Team'])['ScoreType'].count().reset_index()
    run = 0
    for field in dfTD.values:
        if run == 0:
            dfNFLDataForGames.loc[value - 1, 'HomeTouchDowns'] = field[2]
            run = run + 1
        else:
            dfNFLDataForGames.loc[value -1 , 'AwayTouchDowns'] = field[2]
            
            
print 'Scoring Added'                     

Missing Drive Data
Scoring Drives Captured
Scoring Added


In [34]:
#Add passing Columns

columnNames = pd.DataFrame(['PassingComp','PassingAtt','PassingTD','RushingAtt','RushingTD'])

teams = pd.DataFrame(['Home', 'Away'])

for column in columnNames.values:
    insertColumn = column[-1]
    for team in teams.values:
        columnName = team + column
        dfNFLDataForGames[columnName[-1]] = 0

print('Columns Added')

Columns Added


In [35]:
for x in range(0,len(dfAllGames)):
    gameId = dfAllGames.loc[x]['GameID']
    Year = dfAllGames.loc[x]['Year']
    Week = dfAllGames.loc[x]['Week']
    HomeTeam = dfAllGames.loc[x]['HomeTeam']
    AwayTeam = dfAllGames.loc[x]['AwayTeam']

    game = nflgame.one(Year, Week, HomeTeam, AwayTeam)

    try:
        for rush in game.players.rushing().filter(home=True):
            #print(dfRushingDataHome.GameID.count())
            dfRushingData.loc[(dfRushingData.GameID.count() + 1)] =(gameId, HomeTeam, rush.rushing_att, rush.rushing_yds, rush.rushing_tds)

        for rush in game.players.rushing().filter(home=False):
            #print(dfRushingDataHome.GameID.count())
            dfRushingData.loc[(dfRushingData.GameID.count() + 1)] =(gameId, AwayTeam, rush.rushing_att, rush.rushing_yds, rush.rushing_tds)

        #Get Passing Details
        for passer in game.players.passing().filter(home=True):

            dfPassingData.loc[(dfPassingData.GameID.count() + 1)] = (gameId, HomeTeam, passer.passing_cmp, passer.passing_att, passer.passing_yds, passer.passing_tds)

        for passer in game.players.passing().filter(home=False) :
  
            dfPassingData.loc[(dfPassingData.GameID.count() + 1)] = (gameId, AwayTeam, passer.passing_cmp, passer.passing_att, passer.passing_yds, passer.passing_tds)

    except:
        print("Error GameId:" , gameId)

print('Passing and Rushing Totals Created')

('Error GameId:', 1685)
Passing and Rushing Totals Created


# Add Passing Data

In [36]:
dfPassingColumns = pd.DataFrame(['PassingComp', 'PassingAtt', 'PassingTD'])

for value in range(1, maxGameID):
    for column in dfPassingColumns.values:
        insertColumn = column[-1]
        dfPasses = dfPassingData[dfPassingData['GameID'] == value]
        dfPassingStats = dfPasses.groupby(['GameID', 'Team'])[column].sum().reset_index()
        run = 0
        for play in dfPassingStats.values:
            if run == 0:
                dfNFLDataForGames.loc[value - 1, 'Home' + insertColumn] = play[2]
                run = run + 1
            else:
                dfNFLDataForGames.loc[value -1 , 'Away' + insertColumn] = play[2]
                
print('Completed')

Completed


# Add Rushing Data

In [37]:
dfRushingColumns = pd.DataFrame(['RushingAtt', 'RushingTD'])

for value in range(1, maxGameID):
    for column in dfRushingColumns.values:
        insertColumn = column[-1]
        dfRushes = dfRushingData[dfRushingData['GameID'] == value]
        dfRushingStats = dfRushes.groupby(['GameID', 'Team'])[column].sum().reset_index()
        run = 0
        for rush in dfRushingStats.values:
            if run == 0:
                dfNFLDataForGames.loc[value - 1, 'Home' + insertColumn] = rush[2]
                run = run + 1
            else:
                dfNFLDataForGames.loc[value -1 , 'Away' + insertColumn] = rush[2]
                
print('Completed')

Completed


# Add Field Goal Data

In [38]:
for value in range(1, maxGameID):
    dfTeams = dfFieldGoalTotal[dfFieldGoalTotal['GameID'] == value]
    dfFieldGoals = dfTeams.groupby(['GameID', 'Team'])['ScoreType'].count().reset_index()
    run = 0
    for field in dfFieldGoals.values:
        if run == 0:
            dfNFLDataForGames.loc[value - 1, 'HomeFieldGoals'] = field[2]
            run = run + 1
        else:
            dfNFLDataForGames.loc[value -1 , 'AwayFieldGoals'] = field[2]


## Get Time Info 

Created this section so we can parse out different sections of game time data. We want to take a look at if time has any effect on the games outcome. Additionally, we used this data to help with getting game time weather conditions. 

In [39]:
location = 0
dfGameTimes = pd.DataFrame(columns=('GameID', 'Year', 'Week', 'Date', 'Day', 'TimeOfDay', 'NumericDate'))
for year in years:
    for week in weeks:
        future_games = nflgame.live._games_in_week(year, week)
        for fgame in future_games:
            dateOfGame =  '{}/{}/{}'.format(fgame['month'], fgame['day'], year)
            
            if(len(str(fgame['day'])) == 1):
                
                day = '0'+ str(fgame['day'])
            else:
                day = str(fgame['day'])
                
            if(len(str(fgame['month'])) == 1):
                
                month = '0'+ str(fgame['month'])
            
            else:
                month = str(fgame['month'])
            
            
            dfGameTimes.loc[location] = ((location + 1)
                                  , year
                                  , week
                                  , dateOfGame
                                  , datetime.datetime(year, fgame['month'], fgame['day']).weekday()
                                  , fgame['time']
                                  , '{}{}{}'.format(year, month, day))
            
            location = location + 1
            
frames = [dfNFLDataForGames, dfGameTimes]
dfTotalData = pd.concat(frames, axis=1, join='inner')
            
print('Complete')          

Complete


## Clean Weather Data
We removed all -9999 and replace them with NaN. Before joining with the additional data to complete our data set

In [68]:
#remove duplicate columns before Join
dfTotalData = dfTotalData.T.drop_duplicates().T
dfFinal = pd.DataFrame.merge(dfTotalData, dfWeatherInfo, on='GameID', how='left')
#Remove duplicate columns from Merge
dfFinal = dfFinal.T.drop_duplicates().T

#Update -999 with NaN 
updateColumns = pd.DataFrame(['Precipitation' , 'DewPoint' , 'Visiability' , 'Humidity' , 'WindSpeed' 
                              , 'WindDir' , 'Pressure' , 'Fog' , 'Precipitation' , 'Condition' , 'Rain' 
                              , 'Snow' , 'WindChill' , 'HeatIndex'])


for column in updateColumns.values:
    dataColumn = column[-1]
    dfFinal[dataColumn] = dfFinal[dataColumn].replace(-9999, np.nan)
    dfFinal[dataColumn] = dfFinal[dataColumn].replace(-999, np.nan)
    
dfFinal.head()

,GameID,HomeTeam,AwayTeam,GameWinner,GameLoser,ScoreHome,ScoreAway,HomeFirstDowns,HomePassingYards,HomePenaltyCnt,...,WindSpeed,WindDir,Pressure,Fog,Precipitation,Condition,Rain,Snow,WindChill,HeatIndex
0,1,PIT,TEN,PIT,TEN,13,10,19,321,6,...,6.9,ENE,30.28,0.0,NaN,Clear,0.0,0.0,NaN,NaN
1,2,ATL,MIA,ATL,MIA,19,7,19,213,4,...,4.6,East,29.99,0.0,NaN,Clear,0.0,0.0,NaN,NaN
2,3,BAL,KC,BAL,KC,38,24,32,303,8,...,NaN,North,29.95,0.0,NaN,Unknown,0.0,0.0,NaN,NaN
3,4,CAR,PHI,PHI,CAR,10,38,14,83,5,...,4.6,NNE,30.00,0.0,NaN,Mostly Cloudy,0.0,0.0,NaN,81.0
4,5,CIN,DEN,DEN,CIN,7,12,16,221,4,...,5.8,ENE,30.12,0.0,NaN,Clear,0.0,0.0,NaN,NaN


# Create Data File with All Columns

In [69]:
dfFinal.to_csv(directory + 'GameData.csv', sep='|', encoding='utf-8', index=False)  

# Get Weather Data for API

### Do not run this as it techincally violates the weather underground API Rules :-). 

In [55]:
#Import Team Info
#dfTeamInfo = pd.read_csv(directory + 'TeamInfo.csv', sep=',')

#dfWeatherData = dfTotalData[['GameID','Date','NumericDate','TimeOfDay', 'HomeTeam']]

#Did this to remove Duplicate GameID values
#dfWeatherData = dfWeatherData.T.drop_duplicates().T

#dfTeamInfo = dfTeamInfo.rename(columns={'Abbrivation':'HomeTeam'})

#dfTeamLoc = pd.merge(dfWeatherData, dfTeamInfo, on='HomeTeam')

#Create Hour of Game
#dfTeamLoc['Hour'] = dfTeamLoc[['TimeOfDay']]

#i = 0
#for time in dfTeamLoc[['TimeOfDay']].values:
#    time = time[-1]
#    hour = int(time[:1]) + 12
#    dfTeamLoc.loc[i, 'Hour'] = hour
#    i = i + 1

In [57]:
#import time

#dfWeatherAPIData = pd.DataFrame(columns=('GameID', 'Temp', 'DewPoint', 'Visiability', 'Humidity', 'WindSpeed','WindDir'
#                                     , 'Pressure', 'Fog', 'Precipitation', 'Condition', 'Rain', 'Snow', 'WindChill'
#                                     , 'HeatIndex'))

##Create Api String
#apiStart = 'http://api.wunderground.com/api'
#apiKey = 'd929fd774ceb70f4'
#i = 0
#for data in dfTeamLoc.values:
#    gameHour = data[8]
#    gameCity = data[5].replace(' ', '_')
#    gameState = data[6]
#    gameTime = data[2]
#    GameId = data[0]
#    try:
#        #Create API Request
#        apiRequest = '{}/{}/history_{}/q/{}/{}.json'.format(apiStart, apiKey, gameTime, gameState,gameCity)
#        response = requests.get(apiRequest)
#        if(response.status_code == 200):
#           jsonResponse = response.json()
#            observations = jsonResponse['history']['observations']
#            for obs in observations:
#                jsonHour = obs['date']['hour']
#                if(int(jsonHour) == int(gameHour)):
#                    gameTemp = obs['tempi']
#                    gameDewPt = obs['dewpti']
#                    gameVisibility = obs['visi']
#                    gameHumidity = obs['hum']
#                    gameWindSpeed = obs['wspdi']
#                    gameWindDir = obs['wdire']
#                    gamePressure = obs['pressurei']
#                    gameFog = obs['fog']
#                    gameprecip = obs['precipi']
#                    gameConditions = obs['conds']
#                    gameRain = obs['rain']
#                    gameSnow = obs['snow']
#                    gameWindChill = obs['windchilli']
#                    gameHeatIndex = obs['heatindexi']

#                    dfWeatherAPIData.loc[i] = (GameId, gameTemp, gameDewPt, gameVisibility, gameHumidity, gameWindSpeed, gameWindDir
#                                           , gamePressure, gameFog, gameprecip, gameConditions, gameRain, gameSnow
#                                           , gameWindChill, gameHeatIndex)
#        time.sleep(7)
#        i = i + 1
        
#    except:
#        print GameId
#        time.sleep(7)

In [58]:
#dfWeatherAPIData.to_csv(directory + 'APIDataFinal.csv', sep='|', encoding='utf-8', index=False) 